In [5]:
import sys
sys.path.insert(0, r"K:/Thesis/codes/crypto_project")
sys.path.insert(0, r"E:/Thesis/crypto_project")
import os
import sqlite3
import pandas as pd

DATABASE_LOCATION = r"E:/Thesis/database"
from factor_model.model_update.database_generators import (
    RAW_DATA_DB,
    RETURN_DB,
    FACTOR_MODEL_ESTIMATES
)
from typing import Dict


#### 0. Parameters

In [15]:
risk_calculation_parameters = {
    "correlation_half_life": 730,  # days
    "variance_half_life": 365,  # days
    "specific_risk_half_life": 365,
    "date": "2023-03-04"
}

#### 1. Load the factor returns

In [17]:
with sqlite3.connect(os.path.join(DATABASE_LOCATION, FACTOR_MODEL_ESTIMATES)) as conn:
    factor_returns = pd.read_sql_query("SELECT * FROM factor_returns", conn)

In [47]:
from typing import Dict

NON_FACTOR_COLUMNS = ["id", "date", "version_date"]


def get_factor_return_correlation(
    factor_returns: pd.DataFrame, parameters: Dict
) -> pd.DataFrame:
    """
    Calculates the exponentially weighted moving average (EWMA) correlation matrix
    for factor returns based on specified parameters.
    The correlation matrix is only reported for the specified date

    Args:
        factor_returns (pd.DataFrame): DataFrame containing factor returns.
            Columns should include 'date' and the factor returns for each style.
        parameters (Dict): A dictionary containing parameters:
            - 'date': The cob date for factor return estimation.
            - 'correlation_half_life': Half-life for EWMA correlation calculation.

    Returns:
        pd.DataFrame: EWMA correlation matrix for factor returns.
    """
    factor_return_estim = factor_returns[
        factor_returns["date"] <= parameters["date"]
    ].copy()
    style_columns = sorted(
        list(set(factor_return_estim.columns) - set(NON_FACTOR_COLUMNS))
    )
    return (
        factor_return_estim[style_columns]
        .ewm(halflife=parameters["correlation_half_life"])
        .corr()
        .tail(len(style_columns))
    )

In [48]:
correlation = get_factor_return_correlation(factor_returns, risk_calculation_parameters)
display(correlation)

market  momentum  new_coin  reversal      size    volume
1765 market    1.000000  0.020237 -0.227545 -0.048430  0.303360 -0.730431
     momentum  0.020237  1.000000  0.082769 -0.265184 -0.097476  0.053568
     new_coin -0.227545  0.082769  1.000000  0.104668  0.037091  0.406697
     reversal -0.048430 -0.265184  0.104668  1.000000  0.027737  0.041279
     size      0.303360 -0.097476  0.037091  0.027737  1.000000 -0.592027
     volume   -0.730431  0.053568  0.406697  0.041279 -0.592027  1.000000